In [1]:
pip install gradio --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s et

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, ReLU, Dropout, Softmax, Input
from tensorflow.keras.applications.resnet import preprocess_input as preprocess_input_resnet
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_input_inception
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_input_efficientnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet101,VGG16,InceptionV3,EfficientNetB0
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import models
from keras.layers import *
import numpy as np
import os
import io
import pathlib
from glob import glob
import gradio as gr
import tensorflow as tf
from PIL import Image
import requests
from io import BytesIO
from keras.preprocessing.image import *


In [4]:
def build_network_RESNET101(base_model, classes):
    """
    Build a custom neural network architecture by adding fully connected layers on top of a pre-trained base model.

    Parameters:
        base_model (tf.keras.Model): Pre-trained base model without the fully connected layers.
        classes (int): Number of output classes.

    Returns:
        tf.keras.Model: Custom neural network model with the specified architecture.
    """
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(units=1024)(x)
    x = ReLU()(x)
    #x = BatchNormalization()(x)
    x = Dropout(rate=0.75)(x)
    x = Dense(units=classes)(x)
    output = Softmax()(x)
    return output

#Load the weights ResNet101
base_model = ResNet101(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:
    layer.trainable = False

model_output_RESNET101 = build_network_RESNET101(base_model,6)
model1 = Model(base_model.input, model_output_RESNET101)
model1.load_weights('/content/drive/MyDrive/weights models 1-28/fine_tuned_RESNET101_model.h5')


171446536/171446536 [==============================] - 3s 0us/step


In [5]:
def build_network_VGG16(base_model, classes):
    """
    build a new network from a pre-trained model

    parameters:
        -base_model: pretraind model
        -classes: number of classes to classify

    """
    x = Flatten()(base_model.output)

    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)

    x = Dense(units=classes)(x)
    output = Softmax()(x)
    return output

#Load the weights ResNet101
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:
    layer.trainable = False

model_output_VGG16 = build_network_VGG16(base_model,6)
model2 = Model(base_model.input, model_output_VGG16)
model2.load_weights('/content/drive/MyDrive/weights models 1-28/fine_tuned_VGG16_model.h5')


58889256/58889256 [==============================] - 1s 0us/step


In [6]:
def build_network_InceptionV3(base_model, classes):
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(units=256)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=classes)(x)
    output = Softmax()(x)
    return output

#Load the weights ResNet101
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:
    layer.trainable = False

model_output_InceptionV3 = build_network_InceptionV3(base_model,6)
model3 = Model(base_model.input, model_output_InceptionV3)
model3.load_weights('/content/drive/MyDrive/weights models 1-28/fine_tuned_Inception_model.h5')


87910968/87910968 [==============================] - 1s 0us/step


In [7]:
def build_network_EfficientNetB0 (base_model, classes):

    x = Flatten()(base_model.output)
    x = Dense(units=1024)(x)
    x = ReLU()(x)
    #x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    x = Dense(units=classes)(x)
    output = Softmax()(x)
    return output

#Load the weights ResNet101
base_model = EfficientNetB0  (weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

for layer in base_model.layers:
    layer.trainable = False

model_output_EfficientNetB0  = build_network_EfficientNetB0 (base_model,6)
model4 = Model(base_model.input, model_output_EfficientNetB0 )
model4.load_weights('/content/drive/MyDrive/weights models 1-28/fine_tuned_EffecientNET B7_model.h5')


16705208/16705208 [==============================] - 0s 0us/step


In [8]:
def description(area):
  if  area=="Ajloun":
    text="""Ajloun In the northern region lies Ajloun, known for the impressive
            Ajloun Castle,a 12th-century fortress built by the Ayyubids. Exploring
            the castle provides not only historical insights but also stunning views
            of the surrounding landscape."""
    return text
  elif  area=="Jerash":
    text="""Jerash Known for exceptionally well-preserved Roman ruins, Jerash boasts
            an impressive oval-shaped forum, a colonnaded street, and the Temple of Artemis.
            The annual Jerash Festival attracts visitors with cultural performances, adding a
            vibrant touch to the historical site."""
    return text
  elif  area=="Petra":
    text="""Petra, often referred to as the "Rose City," is a UNESCO World Heritage
            Site renowned for its rock-cut architecture and water conduit system.
             Iconic structures like Al-Khazneh andthe Monastery showcase the ancient
             Nabatean civilization's architectural prowess."""
    return text
  elif  area=="UmmQais":
    text="""Umm Qais Nestled in the northwest, Umm Qais offers panoramic views of the
            Sea of Galilee and the Golan Heights. Its ancient Greco-Roman ruins, including
            a well-preserved amphitheater, provide afascinating glimpse into the historical
            tapestry of the region"""
    return text
  elif  area=="RomanAmphitheater":
    text="""Roman Theater Situated in the heart of Amman, the Roman Theater is an ancient amphitheater
            dating back to the 2nd century.Its grandeur and historical significance make it a key
            attraction in the capital, offering visitors a glimpse into Jordan's rich past."""
    return text
  elif  area=="WadiRum":
    text="""Wadi Rum, with its surreal desert landscape, features Martian-like red sand dunes,
            towering cliffs,and ancient petroglyphs. Visitors can embark on camel or jeep tours
            to explore this unique terrain,gaining an appreciation for both its natural beauty
            and historical significance."""
    return text
  else:
    return text

In [9]:
# Define class labels for the drop-down list
class_labels = ["Ajloun", "Jerash", "Petra", "RomanAmphitheater", "UmmQais", "WadiRum"]




# Function to preprocess the image
def preprocess_image(image,model):

    # Convert the image to jpg format
    converted_img_path = f"{image.split('.')[0]}.jpg"
    print(converted_img_path)
    image = Image.open(image)

    if image.mode != 'RGB':
         image = image.convert('RGB')


    image.save(converted_img_path, 'JPEG')

    image = Image.open(converted_img_path)
    image = image.resize((224, 224))
    image =tf.keras.preprocessing.image.img_to_array(image)

    if model=='ResNet101':
        image = image

    elif(model=='VGG16'):
      image = preprocess_input_vgg16(image)

    elif(model=='InceptionV3'):
      image = preprocess_input_inception(image)

    elif(model=='EfficientNetB0'):
      image = preprocess_input_efficientnet(image)

    if(model !='InceptionV3'):
      image /= 255.0
    image = tf.expand_dims(image, axis=0)
    return image

confirm_msg='Please provide more pictures to confirm.'
de=''
# Function to make predictions
def predict_class(image_input,model):
    processed_image = preprocess_image(image_input,model)

    if model=='ResNet101':
        model=model1

    elif(model=='VGG16'):
      model=model2

    elif(model=='InceptionV3'):
      model=model3

    elif(model=='EfficientNetB0'):
      model=model4


    # Make predictions using the model
    predictions = model.predict(processed_image)

    # Get the predicted class label
    predicted_class_index = np.argmax(predictions)
    predicted_class = class_labels[predicted_class_index]
    print(predicted_class)
    # Return the predicted class label

    de=description(predicted_class)

    r=np.max(predictions)

    if round(r,2)<0.7 and round(r,2)>0.4:
      return str(round(r,2))+"I think this is "+predicted_class+"\n"+confirm_msg+"\nAnyway "+de

    elif round(r,2)<0.4:
      erorr_msg="""Sorry, there seems to be an error. It could be due to this picture not
                   representing the archaeological site or being unclear.
                   Could you please share another image for further clarification?"""
      return str(round(r,2))+erorr_msg

    else:
      return "This is "+predicted_class+"\n"+de


#print(predict_class("/content/drive/MyDrive/weights models 1-28/220px-Petra_Treasury.jpg",'ResNet101'))



In [ ]:
import gradio as gr
import os
import time

def print_like_dislike(x: gr.LikeData):
    print(x.index, x.value, x.liked)


def bot(history):
    response = "**That's cool!**"
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.05)
        yield history

def add_text(history, text):
    history = history + [(text, None)]
    return history, gr.Textbox(value="", interactive=False)

filepath=''

def add_file(history, file):
    filepath=file
    history = history + [((file.name,), None)]
    return history


def predict_class_chat(history,btn=filepath,model='ResNet101'):
    response = predict_class(btn,model)
    history[-1][1] = ""
    for character in response:
        history[-1][1] += character
        time.sleep(0.01)
        yield history


with gr.Blocks(mode='blocks',title="tourist guide in Jordan") as demo:

    chatbot = gr.Chatbot(
        value=[[None,'Welcome to your tourist guide! To discover tourist attractions, just upload the picture, and the guide will inform you about the location, provide information, and offer some suggestions.']],
        elem_id="chatbot",
        label="tourist guide in Jordan",
        show_label=True,
        bubble_full_width=False,
        avatar_images=(None, (os.path.join(os.path.dirname('avatar'), "/content/drive/MyDrive/weights models 1-28/avatar.png"))),
    )

    with gr.Row():
        drop_msg=gr.Dropdown(['ResNet101','VGG16','InceptionV3','EfficientNetB0'], label="Select favorite tourist guide",show_label=True,container=True)
        btn = gr.UploadButton("📁 Upload a image", file_types=["image"],label='image')



    drop_msg = drop_msg.input(predict_class_chat, [chatbot, btn,drop_msg], chatbot)

    file_msg = btn.upload(add_file, [chatbot,btn], [chatbot], queue=False).then(
        predict_class_chat, [chatbot, btn], chatbot
    )

    #chatbot.attach_load_event(bot(),every=None)
    chatbot.like(print_like_dislike, None, None)



demo.queue()
demo.launch(debug=True)
print(filepath)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c1bac1b81765df7964.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [11]:
 """custom_css =
.body {
  background: url('/content/66.jpeg');
  background-size: cover;
  height: 100vh;
  display: flex;
  align-items: center;
  justify-content: center;
  margin: 0;
}

.container {
  background-color: rgba(255, 255, 255, 0.5);
  padding: 20px;
  border-radius: 15px;
  text-align: center;
  box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

h1 {
  font-size: 28px;
  color: #2c3e50;
}

p {
  font-size: 18px;
  color: #34495e;
}

#input_image {
  max-width: 100%;
  max-height: 300px;
  border: 2px solid #3498db;
  border-radius: 10px;
  margin-top: 20px;
  box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

input[type="file"] {
  margin-top: 20px;
  padding: 10px;
  border: 1px solid #3498db;
  border-radius: 5px;
}

#selected_class {
  margin-top: 10px;
  padding: 10px;
  border: 1px solid #3498db;
  border-radius: 5px;
}

button {
  margin-top: 10px;
  padding: 12px 24px;
  background-color: rgba(255, 179, 0, 0.7);
  color: white;
  border: none;
  border-radius: 5px;
  cursor: pointer;
}

#prediction_result {
  margin-top: 20px;
  font-weight: bold;
  color: #e74c3c;
}





# Create Gradio Interface
iface = gr.Interface(css=custom_css,theme=gr.themes.Soft(),

    fn=predict_class,
    inputs=[
        gr.Image(type="filepath", label="Upload an image"),
        gr.Dropdown(['ResNet101','VGG16','InceptionV3','EfficientNetB0'], label="Select favorite model")
    ],
    outputs="text",
    live=True,
    title="Archaeological Site Classification Demo",
    description="Upload an image for the archaeological places and get predictions of the name of the archaeological landmark.",

)

# Launch the Gradio Interface
iface.launch()
"""

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1cb680bf74da86a54c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
